In [28]:
from __future__ import print_function
import pandas as pd
import numpy as np

In [3]:
def infoparse(x):
    key   = [i.split('=')[0] for i in x.split(';')]
    value = [i.split('=')[1] for i in x.split(';')]
    return dict(zip(key,value))

In [4]:
def rev_comp(strSeq):
	dicComp = {'A':'T','T':'A','G':'C','C':'G','N':'N'}
	strCseq = ''
	for i in strSeq:
		try:
			strCseq += dicComp[i.upper()]
		except KeyError:
			strCseq += 'N'
	# End of for i
	return(strCseq[::-1])

In [5]:
file_fa = '../../ref/Gmax_275_v2.0.fa'

bulk = open(file_fa).read()
bulk = bulk.split('>')
dicHD2seq = {}
for each_bulk in bulk:
    if each_bulk.strip() == '':
        continue
    genename = each_bulk.split('\n')[0]
    seq      = ''.join(each_bulk.split('\n')[1:])
    dicHD2seq[genename] = seq

In [7]:
file_gff = '../../finalout/my_csv.csv.addgene.gff3.sort.gff3.merge.all.gff3.sort.gff3'

df_gff = pd.read_csv(file_gff,sep='\t',header=None,comment='#')

mask        = (df_gff[2] == 'gene')
df_gff_gene = df_gff[mask]      
df_gff_gene['genename'] = df_gff_gene[8].apply(lambda x:infoparse(x)['ID'])

mask        = (df_gff[2] == 'mRNA')
df_gff_mRNA = df_gff[mask]
mRNA_ID     = df_gff_mRNA[8].apply(lambda x : infoparse(x)['ID'])
mRNA_Parent = df_gff_mRNA[8].apply(lambda x : infoparse(x)['Parent'])
dict2g = dict(zip(mRNA_ID,mRNA_Parent))

mask = (df_gff[2] == 'CDS')
df_gff_cds = df_gff[mask]      

df_gff_cds['genename'] = df_gff_cds[8].apply(lambda x : infoparse(x)['Parent'])

df_gff_cds_index = df_gff_cds.set_index('genename')

df_gff_cds_index = df_gff_cds_index.sort_values(3, ascending=True)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
df_gff_cds_index.head()

,0,1,2,3,4,5,6,7,8
genename,,,,,,,,,
Glyma.U007300.1.Wm82.a2.v1,scaffold_1793,phytozomev10,CDS,8,15,.,+,0,ID=Glyma.U007300.1.Wm82.a2.v1.CDS.1;Parent=Gly...
Gene.156639::STRG.66472.2::g.156639::m.156639,scaffold_486,ST/TD/KYJ,CDS,15,356,.,-,0,ID=cds.Gene.156639::STRG.66472.2::g.156639::m....
Gene.156641::STRG.66472.1::g.156641::m.156641,scaffold_486,ST/TD/KYJ,CDS,15,356,.,-,0,ID=cds.Gene.156641::STRG.66472.1::g.156641::m....
Glyma.U039600.Wm82.a2.v1.st.1,scaffold_484,ST/TD/KYJ,CDS,20,72,.,-,2,ID=cds.Glyma.U039600.Wm82.a2.v1.st.1.4;Parent=...
Glyma.U008600.1.Wm82.a2.v1,scaffold_1990,phytozomev10,CDS,33,160,.,-,2,ID=Glyma.U008600.1.Wm82.a2.v1.CDS.4;Parent=Gly...


In [13]:
i = 0
dic = {'transcriptname' : [],
       'strand' : [],
       'CDSloc' : [],
       'CDSseq' : []}
for genename in set(df_gff_cds_index.index):
    
    dic['transcriptname'].append(genename)
    
    edf = df_gff_cds_index.loc[genename]
    if str(type(edf)) == "<class 'pandas.core.frame.DataFrame'>":
        CDS_list = zip(list(edf[3]),list(edf[4]))
        chromosome = edf[0].values[0]
        strand     = edf[6].values[0]
    else:
        CDS_list = zip([edf[3]],[edf[4]])
        chromosome = edf[0]
        strand     = edf[6]
    
    dic['strand'].append(strand)
    cdsseq = ''
    for l,r in CDS_list:
        #print l,r
        cdsseq += dicHD2seq[chromosome][l-1:r]
    if strand == '+':
        pass
    else: 
        cdsseq = rev_comp(cdsseq)
    dic['CDSloc'].append(CDS_list)
    dic['CDSseq'].append(cdsseq)
    #print genename,chromosome,strand,CDS_list,cdsseq
    #if i == 10:
    #    break
    #i += 1

In [14]:
df_cds = pd.DataFrame(dic)

In [15]:
df_cds['gene'] = df_cds['transcriptname'].apply(lambda x : dict2g[x])

In [16]:
df_cds.head()

,CDSloc,CDSseq,strand,transcriptname,gene
0,<zip object at 0x7fa3e6b7fc48>,ATGGCACTTGCTGAAACAAACATGAAAAGCTCTCTGCAAAATCGCA...,-,Glyma.06G011500.1.Wm82.a2.v1,Glyma.06G011500.Wm82.a2.v1
1,<zip object at 0x7fa3e6b7ff08>,ATGTCTGCGTCCGAAGCGGAGAAGAAGGTCGAGGAGGAAGAGAAGA...,-,Glyma.06G025900.1.Wm82.a2.v1,Glyma.06G025900.Wm82.a2.v1
2,<zip object at 0x7fa3e6b81248>,ATGGCAATTAGTGTTATGAAGGAGGCCAACATGAAAGGGCGTATGG...,-,Glyma.15G228000.1.Wm82.a2.v1,Glyma.15G228000.Wm82.a2.v1
3,<zip object at 0x7fa3e6b7fbc8>,ATGTCTCCTATTCAACTTGGCAGAGGGAAGAATTGGCTAAGATACT...,+,Glyma.18G247600.1.Wm82.a2.v1,Glyma.18G247600.Wm82.a2.v1
4,<zip object at 0x7fa3e6b81348>,ATGAGTAGAAGAAGCATGCCTGCAAAATTGTTGCTCACAACTTTGT...,-,Glyma.14G085400.1.Wm82.a2.v1,Glyma.14G085400.Wm82.a2.v1


In [17]:
df_cds_index = df_cds.set_index(['gene','transcriptname'])

In [18]:
df_cds_index.to_pickle(file_gff+'.cdsseq.pk')

In [24]:
df_cds.head()

,CDSloc,CDSseq,strand,transcriptname,gene
0,<zip object at 0x7fa3e6b7fc48>,ATGGCACTTGCTGAAACAAACATGAAAAGCTCTCTGCAAAATCGCA...,-,Glyma.06G011500.1.Wm82.a2.v1,Glyma.06G011500.Wm82.a2.v1
1,<zip object at 0x7fa3e6b7ff08>,ATGTCTGCGTCCGAAGCGGAGAAGAAGGTCGAGGAGGAAGAGAAGA...,-,Glyma.06G025900.1.Wm82.a2.v1,Glyma.06G025900.Wm82.a2.v1
2,<zip object at 0x7fa3e6b81248>,ATGGCAATTAGTGTTATGAAGGAGGCCAACATGAAAGGGCGTATGG...,-,Glyma.15G228000.1.Wm82.a2.v1,Glyma.15G228000.Wm82.a2.v1
3,<zip object at 0x7fa3e6b7fbc8>,ATGTCTCCTATTCAACTTGGCAGAGGGAAGAATTGGCTAAGATACT...,+,Glyma.18G247600.1.Wm82.a2.v1,Glyma.18G247600.Wm82.a2.v1
4,<zip object at 0x7fa3e6b81348>,ATGAGTAGAAGAAGCATGCCTGCAAAATTGTTGCTCACAACTTTGT...,-,Glyma.14G085400.1.Wm82.a2.v1,Glyma.14G085400.Wm82.a2.v1


In [33]:
m = ['st' in x for x in df_cds['transcriptname']]
df_cds_st = df_cds[m]

In [35]:
m = ['Glyma' not in x for x in df_cds['transcriptname']]
df_cds_new = df_cds[m]

In [36]:
df_cds_new.head()

,CDSloc,CDSseq,strand,transcriptname,gene
39,<zip object at 0x7fa3e6b84348>,AAACTTCATAGTATTGGAAAGAAAGTTGCATACATATTGAAGATAT...,.,Gene.21026::STRG.7013.1::g.21026::m.21026,Gene.21026::STRG
76,<zip object at 0x7fa3e6b960c8>,AAAAATTCACCTCAAAAACAACCAAATAGCAGTAGCAACAATTCAA...,.,Gene.86153::STRG.34756.1::g.86153::m.86153,Gene.86153::STRG
260,<zip object at 0x7fa3e6b98a88>,CAGGAACAACCAGAATTCAACATCCAACCCATACAAATAATTCCTG...,.,Gene.72616::STRG.27688.1::g.72616::m.72616,Gene.72616::STRG
263,<zip object at 0x7fa3e6b25b08>,ATGGTGTTGATCAGAGAAGAAGAGATAGTGGAGGAGATGAGTGAAG...,+,Gene.138207::STRG.56225.2::g.138207::m.138207,Gene.138207::STRG
273,<zip object at 0x7fa3e6b43488>,AAGATGGAAAAATCTGAAAAGAAAAATCATAGTGAGAAGAAACTAA...,+,Gene.149998::STRG.63653.1::g.149998::m.149998,Gene.149998::STRG


In [27]:
with open(file_gff+'.cds.fa','w') as f:
    for ix in df_cds.index:
        print('>'+df_cds.loc[ix]['transcriptname'],file=f)
        print(df_cds.loc[ix]['CDSseq'],file=f)
        

In [37]:
with open(file_gff+'.cds_st.fa','w') as f:
    for ix in df_cds_st.index:
        print('>'+df_cds_st.loc[ix]['transcriptname'],file=f)
        print(df_cds_st.loc[ix]['CDSseq'],file=f)

In [38]:
with open(file_gff+'.cds_new.fa','w') as f:
    for ix in df_cds_new.index:
        print('>'+df_cds_new.loc[ix]['transcriptname'],file=f)
        print(df_cds_new.loc[ix]['CDSseq'],file=f)